In [1]:
import csv
from pathlib import Path

import numpy as np

import coxeter

In [54]:
# Check equivalence between data in coxeter and csv that I made
paths = [
    Path("doc/source/_data/science.1220869.archimedean.csv"),
    Path("doc/source/_data/science.1220869.catalan.csv"),
    Path("doc/source/_data/science.1220869.johnson.csv"),
    Path("doc/source/_data/science.1220869.other.csv"),
    Path("doc/source/_data/science.1220869.platonic.csv"),
]
for file_path in paths:
    with open(file_path, newline="") as infile:
        reader = csv.reader(infile, delimiter=",")

        for i, (id, paper_name) in enumerate(reader):
            if i != 0:
                coxeter_name = coxeter.families.DOI_SHAPE_REPOSITORIES[
                    "10.1126/science.1220869"
                ][0].data[id]["name"]
                if paper_name != coxeter_name:
                    print(
                        f"Name mismatch found for {id}: {paper_name = }, {coxeter_name = }"
                    )

Name mismatch found for J86: paper_name = 'Sphenocorona', coxeter_name = 'Spenocorona'


This is a typo in coxeter - Spenocoronoa should be Sphenocorona. I've updated the data file in coxeter to reflect this.

Create X3D files - note the modified io function to ensure that there is a viewpoint element

In [5]:
from xml.etree import ElementTree


def to_x3d(shape, filename):
    """Save shape to an Extensible 3D (X3D) file.

    Args:
        filename (str, pathlib.Path, or os.PathLike):
            The name or path of the output file, including the extension.

    Raises
    ------
        OSError: If open() encounters a problem.
    """

    # Parent elements
    root = ElementTree.Element(
        "x3d",
        attrib={
            "profile": "Interchange",
            "version": "4.0",
            "xmlns:xsd": "http://www.w3.org/2001/XMLSchema-instance",
            "xsd:schemaLocation": "http://www.web3d.org/specifications/x3d-4.0.xsd",
        },
    )
    x3d_scene = ElementTree.SubElement(root, "Scene")

    # Faces
    x3d_shape_faces = ElementTree.SubElement(
        x3d_scene, "shape", attrib={"DEF": f"{shape.__class__.__name__} Faces"}
    )

    x3d_appearance = ElementTree.SubElement(x3d_shape_faces, "Appearance")
    ElementTree.SubElement(
        x3d_appearance, "Material", attrib={"diffuseColor": "#6495ED"}
    )

    # Faces Geometry data
    point_indices = list(range(sum([len(f) for f in shape.faces])))
    prev_index = 0
    for f in shape.faces:
        point_indices.insert(len(f) + prev_index, -1)
        prev_index += len(f) + 1

    points = [v for f in shape.faces for v_index in f for v in shape.vertices[v_index]]

    x3d_indexedfaceset = ElementTree.SubElement(
        x3d_shape_faces,
        "IndexedFaceSet",
        attrib={"coordIndex": " ".join([str(c_index) for c_index in point_indices])},
    )
    ElementTree.SubElement(
        x3d_indexedfaceset,
        "Coordinate",
        attrib={"point": " ".join([str(p) for p in points])},
    )

    # Outline/Edges
    x3d_shape_edges = ElementTree.SubElement(
        x3d_scene, "shape", attrib={"DEF": f"{shape.__class__.__name__} Edges"}
    )

    x3d_appearance = ElementTree.SubElement(x3d_shape_edges, "Appearance")
    ElementTree.SubElement(
        x3d_appearance, "LineProperties", attrib={"linewidthScaleFactor": "0"}
    )

    # Outline/Edges Geometry data
    point_indices = list(range(sum([len(f) for f in shape.faces])))
    prev_index = 0
    for f in shape.faces:
        point_indices.insert(len(f) + prev_index, -1)
        prev_index += len(f) + 1

    points = [v for f in shape.faces for v_index in f for v in shape.vertices[v_index]]

    x3d_indexedfaceset = ElementTree.SubElement(
        x3d_shape_edges,
        "IndexedLineSet",
        attrib={"coordIndex": " ".join([str(c_index) for c_index in point_indices])},
    )
    ElementTree.SubElement(
        x3d_indexedfaceset,
        "Coordinate",
        attrib={"point": " ".join([str(p) for p in points])},
    )

    # Position the camera
    try:
        camera_pos_z = 3 * shape.circumsphere_radius
    except RuntimeError:
        widths = np.max(shape.vertices, axis=0) - np.min(shape.vertices, axis=0)
        camera_pos_z = (3 / 2) * (np.max(widths))

    x3d_viewpoint = ElementTree.SubElement(
        x3d_scene,
        "Viewpoint",
        attrib={
            "position": f"0,0,{camera_pos_z}",  # Note the hardcoded position
            "centerOfRotation": ",".join(map(str, np.around(shape.centroid, 3))),
        },
    )

    # Write to file
    ElementTree.ElementTree(root).write(filename, encoding="UTF-8")

In [6]:
paths = [
    Path("doc/source/_data/science.1220869.archimedean.csv"),
    Path("doc/source/_data/science.1220869.catalan.csv"),
    Path("doc/source/_data/science.1220869.johnson.csv"),
    Path("doc/source/_data/science.1220869.other.csv"),
    Path("doc/source/_data/science.1220869.platonic.csv"),
]

for file_path in paths:
    with open(file_path, newline="") as infile:
        reader = csv.reader(infile, delimiter=",")

        with open(
            file_path.parent / (file_path.stem + "_processed.csv"), "w", newline=""
        ) as outfile:
            writer = csv.writer(outfile)

            for i, (id, name) in enumerate(reader):
                if i != 0:
                    shape = coxeter.families.DOI_SHAPE_REPOSITORIES[
                        "10.1126/science.1220869"
                    ][0].get_shape(id)
                    newfilepath = Path(
                        "/home/joseph/GlotzerGroup/coxeter/doc/source/_static"
                    ) / (name.lower().replace(" ", "_") + ".x3d")
                    to_x3d(shape, newfilepath)

In [7]:
paths = [
    Path("doc/source/_data/science.1220869.archimedean.csv"),
    Path("doc/source/_data/science.1220869.catalan.csv"),
    Path("doc/source/_data/science.1220869.johnson.csv"),
    Path("doc/source/_data/science.1220869.other.csv"),
    Path("doc/source/_data/science.1220869.platonic.csv"),
]

for file_path in paths:
    with open(file_path, newline="") as infile:
        reader = csv.reader(infile, delimiter=",")

        with open(
            file_path.parent / (file_path.stem + "_processed.csv"), "w", newline=""
        ) as outfile:
            writer = csv.writer(outfile)

            for i, (id, name) in enumerate(reader):
                if i == 0:
                    writer.writerow(["ID", "Name", "Vertices", "Faces", "Model"])
                else:
                    shape = coxeter.families.DOI_SHAPE_REPOSITORIES[
                        "10.1126/science.1220869"
                    ][0].get_shape(id)
                    vertices = shape.num_vertices
                    faces = shape.num_faces
                    model_info = (
                        f":model:`_static/{name.lower().replace(' ', '_')}.x3d`"
                    )
                    writer.writerow([id, name, vertices, faces, model_info])

TODO:
- add collapsibility so that browser is never trying to load all of the x3ds at once
    - if collapsibility does not solve the problem, we might need to try iframes again
        - now that I've figured out how to get the styling for X3D correct, iframes may not be as bad as they were

In [10]:
import coxeter

# Create Square Prism and Triangular Antiprism X3D models
shape = coxeter.families.PrismAntiprismFamily.get_shape("Square Prism")
coxeter.io.to_x3d(shape, "square_prism.x3d")
coxeter.io.to_html(shape, "square_prism.html")

shape = coxeter.families.PrismAntiprismFamily.get_shape("Triangular Antiprism")
coxeter.io.to_x3d(shape, "triangular_antiprism.x3d")
coxeter.io.to_html(shape, "triangular_antiprism.html")

/tmp/ipykernel_1950/1266586993.py:3: DeprecationWarning: PrismAntiprismFamily has been deprecated in favor of UniformPrismFamily and UniformAntiprismFamily. These alternate classes have a simplified interface and support the entire infinite family of geometries.
  shape = coxeter.families.PrismAntiprismFamily.get_shape("Square Prism")


In [6]:
import coxeter

# Create Pyramid and Dipyramid X3D models
names = [
    "Triangular Pyramid",
    "Square Pyramid",
    "Pentagonal Pyramid",
    "Triangular Dipyramid",
    "Square Dipyramid",
    "Pentagonal Dipyramid",
]
for n in names:
    shape = coxeter.families.PyramidDipyramidFamily.get_shape(n)
    print(
        f"{n},{len(shape.vertices)},{len(shape.faces)},:x3d-model:`_static/x3d/{n.lower().replace(' ', '_') + '.x3d'}`"
    )
    # coxeter.io.to_x3d(shape, n.lower().replace(" ", "_") + ".x3d")
    # coxeter.io.to_html(shape, n.lower().replace(" ", "_") + ".html")

Triangular Pyramid,4,4,:x3d-model:`_static/x3d/triangular_pyramid.x3d`
Square Pyramid,5,5,:x3d-model:`_static/x3d/square_pyramid.x3d`
Pentagonal Pyramid,6,6,:x3d-model:`_static/x3d/pentagonal_pyramid.x3d`
Triangular Dipyramid,5,6,:x3d-model:`_static/x3d/triangular_dipyramid.x3d`
Square Dipyramid,6,8,:x3d-model:`_static/x3d/square_dipyramid.x3d`
Pentagonal Dipyramid,7,10,:x3d-model:`_static/x3d/pentagonal_dipyramid.x3d`
